## 03 Decisioning

Copper Day Trader - Daily Directional Trading



### Imports

In [11]:
import numpy as np
import pandas as pd
import ibis
import matplotlib.pyplot as plt
import methods.prep as prep
import methods.vis as vis
import methods.fc as fc
import methods.sim as sim
import importlib

### Load Data

In [47]:
con = ibis.connect("duckdb://")
data = ibis.read_csv('data_forecasting/data.csv')

Dev/Test Split

In [48]:
# Split the data into model development (training + validation) (2007-01-01 to 2019-12-31) 
dev_data = data.filter(data.DATE.year() >= 2007).filter(data.DATE.year() <= 2019)

# and holdout test set (2020-01-01 to the end of the dataset 
# in late 2024, plus 2019 padding to be dropped later)
test_data = data.filter(data.DATE.year() >= 2019)

### Run Forecasting & Enrichment - Development

In [49]:
dev_df = dev_data.to_pandas()
dev_df['DATE'] = pd.to_datetime(dev_df['DATE'])
dev_df = dev_df.sort_values('DATE')

In [50]:
test_df = test_data.to_pandas()
test_df['DATE'] = pd.to_datetime(test_df['DATE'])
test_df = test_df.sort_values('DATE')

In [51]:
# Forecast
importlib.reload(fc)
dev_df = fc.sliding_window_arima_predictions(
    df = dev_df,
    target_name= 'COPPER_OPEN_NOMINAL',
    pdq = (1,2,1),
    window_size=12)

C:\Users\haits\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\haits\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
C:\Users\haits\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:

In [53]:
# Enrichment
# Add several columns based on the prediction target
# to aid in evaluation and decisioning
importlib.reload(fc)
dev_df = fc.add_fc_eval_columns(
    df = dev_df,
    pred_feature = 'COPPER_OPEN_NOMINAL')

Creat buckets for predicted price deltas for the decisioning model to act on

In [ ]:
# Calculate 10 breakpoints to divide predicted deltas into 11 buckets
breakpoints = list(dev_df['COPPER_OPEN_NOMINAL_DELTA_PRED'].quantile([x/11.0 for x in range(1,11)]))
breakpoints

[-0.0311147046426547,
 -0.018560733261746343,
 -0.011318009992273958,
 -0.006399859374707036,
 -0.002237556528396422,
 0.0017919424942032843,
 0.0061206536604125035,
 0.011921092522014799,
 0.019527357417301085,
 0.03370639757143934]

Limit to 2008-2019 due to NaNs; drop remaining NaN

In [77]:
dev_df = dev_df[dev_df['DATE'].dt.year >= 2008].dropna()

In [78]:
def calc_bin(x : float,
             breakpoints : list)->int:

    bin = 0
    for breakpoint in breakpoints:
        if x < breakpoint:
            return bin
        bin = bin + 1
    return bin

In [79]:
def bins_by_breakpoint(
        df : pd.DataFrame,
        breakpoints : list,
        target_col : str)->pd.DataFrame:
    
    df[target_col + '_BINS'] = df[target_col].apply(lambda x : calc_bin(x, breakpoints))
    return df

In [80]:
dev_df = bins_by_breakpoint(dev_df,
    breakpoints,
    'COPPER_OPEN_NOMINAL_DELTA_PRED',)

In [81]:
dev_df[['DATE','COPPER_OPEN_NOMINAL_DELTA_PRED','COPPER_OPEN_NOMINAL_DELTA_PRED_BINS']]

,DATE,COPPER_OPEN_NOMINAL_DELTA_PRED,COPPER_OPEN_NOMINAL_DELTA_PRED_BINS
252,2008-01-02,-0.071276,0
253,2008-01-03,-0.041131,0
254,2008-01-04,-0.000145,5
255,2008-01-07,0.097101,10
256,2008-01-08,0.045492,10
...,...,...,...
3291,2019-12-24,0.007298,7
3292,2019-12-25,0.003538,6
3293,2019-12-26,0.003721,6
3294,2019-12-27,0.006904,7
